In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.preprocessing
import sklearn.linear_model

%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 13.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Help see things more compactly
np.set_printoptions(precision=4)

In [2]:
# See https://web.stanford.edu/~hastie/ElemStatLearn/datasets/prostate.info.txt
prostate_raw = pd.read_csv("prostate-cancer.tsv", sep='\t')
# Put the response column first for convenience
prostate_raw = prostate_raw[["lpsa", "lcavol", "lweight", "age", "lbph", "svi", "lcp", "gleason", "pgg45", "train"]]

#prostate_train_raw = prostate_raw.loc[prostate_raw["train"] == "T"].drop(columns=["train"])
#prostate_test_raw  = prostate_raw.loc[prostate_raw["train"] == "F"].drop(columns=["train"])

#scatter_matrix_plot = pd.plotting.scatter_matrix(prostate_train_raw)

In [3]:
#standardized_prostate = ((prostate - prostate.mean())/prostate.std())

scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(prostate_train_raw)  # computes variance dividing by n, not n - 1 
prostate_train = scaler.transform(prostate_train_raw)

In [4]:
n = prostate_train.shape[0]
cov = prostate_train.T.dot(prostate_train) / n
# Print table 3.1 from the book
cov[2:, 1:-1]

array([[ 0.3002,  1.    ,  0.3167,  0.437 ,  0.1811,  0.1568,  0.0236],
       [ 0.2863,  0.3167,  1.    ,  0.2873,  0.1289,  0.173 ,  0.3659],
       [ 0.0632,  0.437 ,  0.2873,  1.    , -0.1391, -0.0885,  0.033 ],
       [ 0.5929,  0.1811,  0.1289, -0.1391,  1.    ,  0.6712,  0.3069],
       [ 0.692 ,  0.1568,  0.173 , -0.0885,  0.6712,  1.    ,  0.4764],
       [ 0.4264,  0.0236,  0.3659,  0.033 ,  0.3069,  0.4764,  1.    ],
       [ 0.4832,  0.0742,  0.2758, -0.0304,  0.4814,  0.6625,  0.7571]])

In [8]:
#X = prostate_train[:, 1:] # X_tr
#y = prostate_train[:, 0]  # y_tr
yX_combined = prostate_raw.drop(columns=["train"]).to_numpy()
X = yX_combined[:, 1:]
y = yX_combined[:, 0]

print(X)
print(y)

XTXinv = np.linalg.inv(X.T.dot(X))
#print(np.diagonal(XTXinv))

B = XTXinv.dot(X.T.dot(y))
B

[[-5.7982e-01  2.7695e+00  5.0000e+01 -1.3863e+00  0.0000e+00 -1.3863e+00
   6.0000e+00  0.0000e+00]
 [-9.9425e-01  3.3196e+00  5.8000e+01 -1.3863e+00  0.0000e+00 -1.3863e+00
   6.0000e+00  0.0000e+00]
 [-5.1083e-01  2.6912e+00  7.4000e+01 -1.3863e+00  0.0000e+00 -1.3863e+00
   7.0000e+00  2.0000e+01]
 [-1.2040e+00  3.2828e+00  5.8000e+01 -1.3863e+00  0.0000e+00 -1.3863e+00
   6.0000e+00  0.0000e+00]
 [ 7.5142e-01  3.4324e+00  6.2000e+01 -1.3863e+00  0.0000e+00 -1.3863e+00
   6.0000e+00  0.0000e+00]
 [-1.0498e+00  3.2288e+00  5.0000e+01 -1.3863e+00  0.0000e+00 -1.3863e+00
   6.0000e+00  0.0000e+00]
 [ 7.3716e-01  3.4735e+00  6.4000e+01  6.1519e-01  0.0000e+00 -1.3863e+00
   6.0000e+00  0.0000e+00]
 [ 6.9315e-01  3.5395e+00  5.8000e+01  1.5369e+00  0.0000e+00 -1.3863e+00
   6.0000e+00  0.0000e+00]
 [-7.7653e-01  3.5395e+00  4.7000e+01 -1.3863e+00  0.0000e+00 -1.3863e+00
   6.0000e+00  0.0000e+00]
 [ 2.2314e-01  3.2445e+00  6.3000e+01 -1.3863e+00  0.0000e+00 -1.3863e+00
   6.0000e+00  0.

array([ 0.5616,  0.6363, -0.0208,  0.0939,  0.7636, -0.1066,  0.0652,
        0.0042])

In [9]:
lm = sklearn.linear_model.LinearRegression().fit(X, y)
lm.coef_

array([ 0.5643,  0.622 , -0.0212,  0.0967,  0.7617, -0.1061,  0.0492,
        0.0045])

In [11]:
p = X.shape[1]            # does not include mean
p

8